In [1]:
import numpy as np
import evaluate
import nltk
import numpy as np
import sentencepiece
import pandas as pd
import transformers
import evaluate
import torch
import os
from rank_bm25 import BM25Okapi
import pandas as pd
import numpy as np
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

2023-07-07 00:18:55.988914: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-07 00:18:56.569475: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
vk = 'test_dev/test_dev_'

vk_qrels = pd.read_csv(vk + 'qrels.tsv', names=['id', 'query_id', 'doc_id'],  sep='\t')
vk_docs = pd.read_csv(vk + 'docs.tsv', names=['id', 'doc_id', 'data'],  sep='\t')
vk_queries = pd.read_csv(vk + 'queries.tsv', names=['id', 'query_id', 'data'],  sep='\t')

In [3]:
def create_joined_file(df_docs, df_qrels, df_queries, path_processed_joined=None):
    joined_df = df_qrels.merge(df_queries, on='query_id').merge(df_docs, on='doc_id', how='left')[['query_id', 'data_x', 'doc_id', 'data_y']]
    joined_df.rename(columns={'data_x':'query_data', 'data_y':'doc_data'}, inplace=True)
    if path_processed_joined:
        joined_df.to_csv(path_processed_joined, sep='\t', index=None, header=None)
    return joined_df

def preprocess_function(dataset):
    docs = [str(i) for i in dataset['doc_data'].values]
    lbls = [str(i) for i in dataset['query_data'].values]
    inputs = tokenizer(docs, max_length=max_input_length, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(lbls, max_length=max_target_input, truncation=True)
    inputs['labels'] = labels['input_ids']
    return inputs

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels!= - 100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    decoded_preds = ['\n'.join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ['\n'.join(nltk.sent_tokenize(pred.strip())) for pred in decoded_labels]
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: v * 100 for k, v in result.items()}
    
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result['gen_len'] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}

vk_joined = create_joined_file(vk_docs, vk_qrels, vk_queries)

In [4]:
len(vk_qrels), len(vk_docs) #, len(ms_qrels), len(ms_docs), len(vk_joined)

(15857, 15857)

In [5]:
vk_joined[['query_id', 'query_data', 'doc_id', 'doc_data']].head(3)

,query_id,query_data,doc_id,doc_data
0,0,0 00 дом muzono net raim feat artur adil скачать,D8,показать меню главная узбекские песни зарубежн...
1,1,0 16 0 8t t,D15,войти на сайт главная задать вопрос представьт...
2,2,0 5 текила максимо де кодорниз сильвер 38,D28,280 другие спиртные напитки с содержанием этил...


### Train-Test split

In [6]:
MODEL_NAME = '/home/tatiana/MADE/Project/VK_Dataset/rut5-base-absum-finetuned-rut513000_3_epochs/checkpoint-31500'
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

dataset = vk_joined[['doc_data', 'query_data']]
dataset = dataset.sample(frac=1).reset_index(drop=True)
train_size = int(len(dataset) * 0.8)
train_dataset = dataset[:train_size].reset_index(drop=True)
test_dataset = dataset[train_size:].reset_index(drop=True)

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {}
        item['input_ids'] = self.encodings[idx]
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels) 

### Rouge

In [9]:
metric = evaluate.load('rouge')

### Train

In [10]:
train_dataset.shape

(12685, 2)

In [11]:
model.cuda()
max_input_length = 400
max_target_input = 100

tokenized_train_dataset = preprocess_function(train_dataset)
tokenized_test_dataset = preprocess_function(test_dataset)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
training_set = Dataset(tokenized_train_dataset['input_ids'], tokenized_train_dataset['labels'])
validation_set = Dataset(tokenized_test_dataset['input_ids'], tokenized_test_dataset['labels'])

/home/tatiana/anaconda3/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3606: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [12]:
batch_size = 2
model_name = MODEL_NAME.split('/')[-1]

EXPERIMENT = '31500_5_epochs'

args = Seq2SeqTrainingArguments(
    f'{model_name}-finetuned-rut5' + EXPERIMENT,
    evaluation_strategy='epoch', 
    learning_rate=2e-5, 
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=1, 
    num_train_epochs=5, 
    predict_with_generate=True, 
    fp16=False,
    push_to_hub=False    
)

nltk.download('punkt')

torch.cuda.empty_cache()

trainer = Seq2SeqTrainer(
    model, 
    args, 
    train_dataset=training_set,
    eval_dataset=validation_set,
    data_collator = data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

### Дообучение модели в течение 5 эпох  

In [13]:
nltk.download('punkt')

torch.cuda.empty_cache()

trainer = Seq2SeqTrainer(
    model, 
    args, 
    train_dataset=training_set,
    eval_dataset=validation_set,
    data_collator = data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

[nltk_data] Downloading package punkt to /home/tatiana/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/home/tatiana/anaconda3/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,3.094200,2.777075,7.893500,1.752000,7.856800,7.828500,10.213100
2,2.946800,2.732403,8.335900,1.883300,8.295000,8.258200,10.764800
3,2.809100,2.718696,8.178800,1.691400,8.158100,8.136400,10.869800
4,2.690900,2.716847,8.737500,1.881300,8.692500,8.673200,11.035000
5,2.651600,2.718578,8.582100,1.784900,8.572000,8.565000,10.882400


TrainOutput(global_step=31715, training_loss=2.895531035304051, metrics={'train_runtime': 8635.8651, 'train_samples_per_second': 7.344, 'train_steps_per_second': 3.672, 'total_flos': 3.352394575291392e+16, 'train_loss': 2.895531035304051, 'epoch': 5.0})

In [26]:
prediction = trainer.predict(test_dataset=validation_set)
prediction.predictions.shape

preds, labels = prediction.predictions, prediction.label_ids
decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
labels = np.where(labels!= - 100, labels, tokenizer.pad_token_id)
decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
decoded_preds = ['\n'.join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
decoded_labels = ['\n'.join(nltk.sent_tokenize(pred.strip())) for pred in decoded_labels]

### Предсказание дообученной и сохраненной моделью

## Prediction ON DEV

In [14]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, T5Tokenizer, Seq2SeqTrainer

torch.cuda.empty_cache()
model = AutoModelForSeq2SeqLM.from_pretrained('rut5-base-absum-finetuned-rut513000_3_epochs/checkpoint-31500')
tokenizer = T5Tokenizer.from_pretrained('rut5-base-absum-finetuned-rut513000_3_epochs/checkpoint-31500')
batch_size = 1
args = Seq2SeqTrainingArguments(
    'rut5-base-absum-finetuned-rut5-100k_docs_2',
    evaluation_strategy='epoch', 
    learning_rate=2e-5, 
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=1, 
    num_train_epochs=1, 
    predict_with_generate=True, 
    fp16=False,
    push_to_hub=False    
)
trainer = Seq2SeqTrainer(
    model, 
    args, 
    train_dataset=training_set,
    eval_dataset=validation_set,
    data_collator = data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [15]:
max_input_length = 400
max_target_input = 400

dev_qrels = pd.read_csv('dev_docs/' + 'qrels.tsv', names=['id', 'query_id', 'doc_id'],  sep='\t')
dev_docs = pd.read_csv('dev_docs/' + 'docs.tsv', names=['id', 'doc_id', 'data'],  sep='\t')
dev_queries = pd.read_csv('dev_docs/' + 'queries.tsv', names=['id', 'query_id', 'data'],  sep='\t')

In [16]:
joined_df = create_joined_file(dev_docs, dev_qrels, dev_queries)
pred_dataset = joined_df[['doc_data', 'query_data']]
pred_tokenized = preprocess_function(pred_dataset)

prediction_set = Dataset(pred_tokenized['input_ids'], pred_tokenized['labels'])
prediction = trainer.predict(test_dataset=prediction_set)

preds, labels = prediction.predictions, prediction.label_ids
decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
labels = np.where(labels!= - 100, labels, tokenizer.pad_token_id)
decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
decoded_preds = ['\n'.join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
decoded_labels = ['\n'.join(nltk.sent_tokenize(pred.strip())) for pred in decoded_labels]

/home/tatiana/anaconda3/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3606: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [17]:
joined_df['preds'] = decoded_preds

In [18]:
joined_df.head(10)

,query_id,query_data,doc_id,doc_data,preds
0,0,0 05 diamonds rihanna скачать,D10,музофил rihanna diamonds diamonds исполнитель ...,скачать песню риганна
1,23,1 13 2 шейдеры,D487,﻿ текущая версия minecraft 116 шейдеры для min...,скачать шейдер для майнкрафт 1 6 2
2,37,1 детская поликлиника в спб,D818,с‡рёс‚р°р№с‚рµ рѕр°сѓ рњрѕсѓрєрір° рё рњрћ рўр...,рё рё рё рё рё рё
3,41,1 й драгунский полк,D944,﻿ загрузка скачать московский 1й лейбдрагунски...,московский 1й лейб драгунский полк
4,43,1 канал в эфире,D990,онлайн телевидение смотри самые популярные кан...,смотреть первый канал онлайн бесплатно
5,46,1 комнатная квартира в шилово,D1072,﻿ п шилово рязанская область специалисты вход ...,купить квартиру в рязани в аренду в москве в м...
6,55,1 псч фгку 21 отряд фпс по хабаровскому краю,D1315,вход rusen русский english фгку 21 отряд фпс п...,фпс хабаровск
7,86,1001 мелоч нижний новгород,D2124,перейти к содержимому 1001 мелочь магазин хозя...,купить мягкие и мягкие мягкие мягкие и
8,91,103 расписание смоленск,D2201,расписание автобуса смоленск – плембаза распис...,расписание автобусов смоленск плембаза
9,92,105 закон калининградской области земельные уч...,D2238,закон калининградской области об особенностях ...,закон об особенностях регулирования земельных ...


In [19]:
joined_df['old_doc_data'] = joined_df['doc_data']
joined_df['doc_data'] = joined_df['preds'] + ' ' + joined_df['doc_data']
joined_df.head(3)

,query_id,query_data,doc_id,doc_data,preds,old_doc_data
0,0,0 05 diamonds rihanna скачать,D10,скачать песню риганна музофил rihanna diamonds...,скачать песню риганна,музофил rihanna diamonds diamonds исполнитель ...
1,23,1 13 2 шейдеры,D487,скачать шейдер для майнкрафт 1 6 2 ﻿ текущая в...,скачать шейдер для майнкрафт 1 6 2,﻿ текущая версия minecraft 116 шейдеры для min...
2,37,1 детская поликлиника в спб,D818,рё рё рё рё рё рё с‡рёс‚р°р№с‚рµ рѕр°сѓ рњрѕсѓ...,рё рё рё рё рё рё,с‡рёс‚р°р№с‚рµ рѕр°сѓ рњрѕсѓрєрір° рё рњрћ рўр...


### Посчитаем скор на документах + сгенерированных дообученной моделью запросах

In [20]:
import os
from rank_bm25 import BM25Okapi
import pandas as pd
import numpy as np

corpus = list(joined_df['doc_data'].values)
tokenized_corpus = [str(corp).split(" ") for corp in corpus]
bm25 = BM25Okapi(tokenized_corpus)

def predicting_100_top_doc_id(joined_df, bm25):
    predict_on_query = []
    predicted_scores = []
    for query_id, query in zip(list(joined_df['query_id']), list(joined_df['query_data'])):
        query = query.split()
        score = bm25.get_scores(query)
        top_100_scores = np.round(score[score >= pd.Series(score).nlargest(100).values[-1]], 1)# score[score >= pd.Series(score).nlargest(100).values[-1]]
        top_100_doc_id = joined_df['doc_id'][score >= pd.Series(score).nlargest(100).values[-1]].values
        score_dicts = [{doc:score} for (doc, score) in zip(top_100_doc_id, top_100_scores)]
        predict_on_query.append(list(top_100_doc_id))
        predicted_scores.append(list(top_100_scores))
    return predict_on_query, predicted_scores

joined_df['predict_100'], joined_df['predict_100_score'] = predicting_100_top_doc_id(joined_df, bm25)
joined_df.head(3)

,query_id,query_data,doc_id,doc_data,preds,old_doc_data,predict_100,predict_100_score
0,0,0 05 diamonds rihanna скачать,D10,скачать песню риганна музофил rihanna diamonds...,скачать песню риганна,музофил rihanna diamonds diamonds исполнитель ...,"[D10, D2394, D9849, D16322, D18018, D18794, D2...","[38.3, 5.9, 5.2, 10.9, 5.2, 8.1, 5.7, 5.4, 5.5..."
1,23,1 13 2 шейдеры,D487,скачать шейдер для майнкрафт 1 6 2 ﻿ текущая в...,скачать шейдер для майнкрафт 1 6 2,﻿ текущая версия minecraft 116 шейдеры для min...,"[D487, D7056, D18377, D20912, D45962, D47103, ...","[20.9, 11.4, 10.6, 10.8, 10.3, 10.5, 10.5, 10...."
2,37,1 детская поликлиника в спб,D818,рё рё рё рё рё рё с‡рёс‚р°р№с‚рµ рѕр°сѓ рњрѕсѓ...,рё рё рё рё рё рё,с‡рёс‚р°р№с‚рµ рѕр°сѓ рњрѕсѓрєрір° рё рњрћ рўр...,"[D9984, D49696, D56674, D79257, D85773, D88617...","[15.4, 13.7, 13.2, 22.9, 16.1, 13.1, 14.9, 14...."


In [59]:
from rank_eval import Qrels, Run, evaluate
qrels = Qrels()
qrels.add_multi(q_ids=[str(i) for i in list(joined_df['query_id'].values)], 
                doc_ids=[[i] for i in joined_df['doc_id']],
                scores=[[1.0] for i in range(len(joined_df))])

run = Run()
run.add_multi(q_ids=[str(i) for i in list(joined_df['query_id'].values)], 
                doc_ids=[i for i in joined_df['predict_100']],
                scores=[i for i in joined_df['predict_100_score']]) # [[1.0] * 100 for i in range(len(joined_df))])

In [60]:
evaluate(qrels, run, ["mrr@100"])

0.7634357665450471

### 0.7628803417941733 on dev 2 epochs with 30k docs
### 0.7613182061766149 on dev 1 epoch with 50k docs